In [1]:
%cd vision_models/

/projects/dali/mayanks4/hessian-spectrum/vision_models


In [2]:
from models.fcn import FCN
model = FCN(in_features=2,out_features=1)
model = model.cuda()

In [3]:
import pickle as pkl
import torch

In [4]:
from torch.utils.data import Dataset,DataLoader

In [5]:
with open("gauss2.pkl","rb") as f:
    gauss_data = pkl.load(f)

/projects/dali/mayanks4/envs/fl/lib/python3.8/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [6]:
class Gauss2(Dataset):
    """Gauss2 two classes"""

    def __init__(self, dataset,  transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample = self.dataset[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample

In [7]:
import hessian_spectrum

In [8]:
argsuse_minibatch = True
gradient_accumulation_steps = 10
batch_size = 1
comment = ''
pred_hessian = True

In [9]:
train_dataset = Gauss2(gauss_data)
train_loader = DataLoader(train_dataset,
                            batch_size=batch_size,
                            shuffle=False)

In [10]:
sample_layer = ['fc.weight']
load_iter = 0

In [11]:
model.fc.weight = torch.nn.Parameter(torch.Tensor([[2, 3]]).to('cuda:0'), requires_grad=True)
model.fc.bias = torch.nn.Parameter(torch.Tensor([4]).to('cuda:0'), requires_grad=True)


In [12]:
train_loader =  [(torch.tensor([1., 2.], device='cuda:0').reshape(1,-1), torch.Tensor([1.]))]


In [13]:
hessian = hessian_spectrum.Hessian(model = model,
                                ckpt_iteration = load_iter,
                                use_minibatch= argsuse_minibatch,
                                gradient_accumulation_steps = gradient_accumulation_steps, 
                                train_data = train_loader,
                                batch_size= batch_size,
                                sample_layer = sample_layer, 
                                comment = comment, 
                                pred_hessian=False,
                                with_ell_prime=False)

total batch 1
name = fc.weight, #params = 2
name = fc.bias, #params = 1
total params 3


In [14]:
hessian.get_spectrum(layer_by_layer=True)

current k 0


/projects/dali/mayanks4/envs/fl/lib/python3.8/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
train_loader

In [87]:
for x,y in train_loader:
    print(x)
    print(y)

tensor([1., 2.], device='cuda:0')
tensor([1])


In [17]:
hessian.load_curve(layer_by_layer=True)

curve [0. 0. 0. ... 0. 0. 0.]


In [15]:
model.fc.bias

Parameter containing:
tensor([-0.2582, -0.0816], device='cuda:0', requires_grad=True)

In [52]:
gauss_data[0][1].dtype

AttributeError: 'int' object has no attribute 'dtype'

[(tensor([1., 2.], device='cuda:0'), 1)]

In [71]:
X,Y = next(iter(train_loader))

In [72]:
model(X)

tensor([144., 361.], device='cuda:0', grad_fn=<PowBackward0>)

In [73]:
criterion = torch.nn.NLLLoss()

In [75]:
criterion(model(X.cuda()),Y.cuda())

RuntimeError: size mismatch (got input: [2], target: [1])

In [77]:
model(X.cuda())

tensor([144., 361.], device='cuda:0', grad_fn=<PowBackward0>)

In [79]:
pred = model(X.reshape(1,-1))

In [82]:
criterion(pred,Y.cuda())

tensor(-361., device='cuda:0', grad_fn=<NllLossBackward0>)